In [1]:
%reload_ext autoreload
%autoreload 2

In [5]:
from depsurf import DepKind
from depsurf.output import load_df

NUM_PROG_COL = ("#Programs",) * 3

df = load_df("dep")
df[NUM_PROG_COL] = df["Program"].apply(len)
df = df.drop(columns=["Program"], level=0)
df = df.sort_values(by=["Type", NUM_PROG_COL], ascending=False)

# df["Status"][df.index.get_level_values("Type") == DepKind.FUNC]
# df[df.index.get_level_values("Type") == DepKind.FIELD]

[           pd.py:53 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/dep.pkl


In [4]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

from depsurf import (
    BaseDepCell,
    DepKind,
    Versions,
    DepStatusEnum,
    DepDeltaEnum,
)
from depsurf.output import save_fig

KEYS = [
    (DepKind.FUNC, "tcp_v6_connect"),
    (DepKind.FUNC, "blk_account_io_start"),
    (DepKind.FUNC, "blk_account_io_done"),
    # (DepKind.FUNC, "blk_account_io_merge_bio"),
    (DepKind.FUNC, "vfs_open"),
    (DepKind.FUNC, "vfs_read"),
    (DepKind.FUNC, "vfs_fsync"),
    (DepKind.FUNC, "__page_cache_alloc"),
    # (DepKind.FUNC, "mark_buffer_dirty"),
    (DepKind.FUNC, "__blk_account_io_done"),
    (DepKind.FUNC, "__blk_account_io_start"),
    (DepKind.TRACEPOINT, "block_rq_complete"),
    (DepKind.TRACEPOINT, "kmalloc_node"),
    (DepKind.TRACEPOINT, "kmem_cache_alloc"),
    (DepKind.TRACEPOINT, "percpu_alloc_percpu"),
    (DepKind.TRACEPOINT, "writeback_dirty_page"),
    (DepKind.TRACEPOINT, "writeback_dirty_folio"),
    (DepKind.STRUCT, "task_struct"),
    (DepKind.STRUCT, "sock"),
    (DepKind.STRUCT, "gendisk"),
    (DepKind.STRUCT, "request"),
    (DepKind.STRUCT, "file"),
    (DepKind.FIELD, "request::cmd_flags"),
    (DepKind.FIELD, "request::rq_disk"),
    (DepKind.FIELD, "request_queue::disk"),
    (DepKind.FIELD, "task_struct::__state"),
    (DepKind.FIELD, "task_struct::state"),
    (DepKind.FIELD, "bio::bi_bdev"),
    (DepKind.FIELD, "bio::bi_disk"),
    (DepKind.FIELD, "renamedata::new_mnt_idmap"),
    (DepKind.FIELD, "renamedata::old_mnt_userns"),
]


df = df.loc[KEYS]
fig, ax = plt.subplots(figsize=(9, 6))
ax: plt.Axes

data = np.array(
    [
        [
            val.background_color if isinstance(val, BaseDepCell) else (1.0, 1.0, 1.0)
            for val in row
        ]
        for row in df.values
    ]
)

cols = df.columns.get_level_values(2)
ax.set_xticks(np.arange(data.shape[1]), labels=cols, rotation=90, fontsize=9)
ax.set_yticks(np.arange(data.shape[0]), labels=[n for t, n in KEYS], fontsize=9)
ax.set_xticks(np.arange(data.shape[1] + 1) - 0.5, minor=True)
ax.set_yticks(np.arange(data.shape[0] + 1) - 0.5, minor=True)
ax.imshow(data)

ax.grid(which="minor", color="white", linestyle="-", linewidth=0.1)
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
ax.spines["top"].set_visible(True)
ax.spines["bottom"].set_visible(False)
ax.tick_params(length=0)


def plot_secondary_ticks(ax: plt.Axes, axis_name, lengths, labels, pad, rotation=0):
    tick_vals = np.array(list(lengths))
    tick_cum = np.cumsum(tick_vals)
    if axis_name == "x":
        axis = ax.secondary_xaxis("top").xaxis
    else:
        axis = ax.secondary_yaxis("left").yaxis

    # plot labels
    axis.set_ticks(
        ticks=tick_cum - tick_vals / 2 - 0.5,
        labels=labels,
        rotation=rotation,
        verticalalignment="center",
        horizontalalignment="center",
    )
    axis.set_tick_params(length=0, pad=pad)

    # plot ticks
    axis.set_ticks(
        ticks=np.insert(tick_cum, 1, 0) - 0.5,
        labels=[],
        minor=True,
    )
    axis.set_tick_params(length=pad + 10, width=0.75, which="minor")

    axline = ax.axvline if axis_name == "x" else ax.axhline
    for d in tick_cum:
        axline(d - 0.5, color="black", lw=0.75)


major_col = defaultdict(int)
for s, g, v in df.columns:
    if isinstance(g, Versions):
        major_col[(s, g)] += 1

plot_secondary_ticks(
    ax, "x", major_col.values(), [f"{g}" for s, g in major_col.keys()], pad=60
)


major_row = defaultdict(int)
for t, n in df.index:
    major_row[t] += 1

plot_secondary_ticks(
    ax,
    "y",
    major_row.values(),
    major_row.keys(),
    pad=125,
    rotation=90,
)


for i, row in enumerate(df.values):
    for j, val in enumerate(row):
        ax.text(
            j,
            i,
            val.text if hasattr(val, "text") else val,
            ha="center",
            va="center",
            color=val.text_color if hasattr(val, "text_color") else "black",
            fontsize=8,
        )

enums = list(DepStatusEnum) + [
    DepDeltaEnum.REMOVE,
    DepDeltaEnum.ADD,
    DepDeltaEnum.CHANGE,
]


legends = [(f"{e.symbol}: {e.name}", e.color) for e in enums]
ax.legend(
    handles=[plt.Rectangle((0, 0), 1, 1, fc=color) for text, color in legends],
    labels=[text for text, color in legends],
    loc="upper left",
    bbox_to_anchor=(-0.29, 1.225),
    fontsize=8,
    ncol=2,
    handlelength=0.8,
    handleheight=0.8,
    handletextpad=0.25,
    columnspacing=0.5,
)


save_fig(fig, "dep")

[          mpl.py:78 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/dep.pdf
